In [1]:
import pandas as pd
import boto3
import json
import pandas as pd
import io

# the below extension properly formats a cell after it is run
%load_ext nb_black

# Set the maximum number of rows to 200
pd.set_option("display.max_rows", 200)
import pandas as pd

# Set the maximum number of columns to 200
pd.set_option("display.max_columns", 200)

<IPython.core.display.Javascript object>

In [2]:
# Creating an S3 client object
s3 = boto3.client('s3')

# Specifying the name of the bucket
bucket_name = 'capstonehaystacks'

# Downloading the all_zips_grocery_store.json file from S3 and parsing it into data1
response1 = s3.get_object(Bucket=bucket_name, Key='all_zips_grocery_store.json')
json_content1 = response1['Body'].read().decode('utf-8')
grocery_json = json.loads(json_content1)

# Downloading the all_zips_restaurant.json file from S3 and parsing it into data2
response2 = s3.get_object(Bucket=bucket_name, Key='all_zips_restaurant.json')
json_content2 = response2['Body'].read().decode('utf-8')
restaurant_json = json.loads(json_content2)


<IPython.core.display.Javascript object>

In [3]:
# THIS CODE LOADS POI'S WITH NO RATINGS AND ALSO INCLUDES ADDRESS AND ZIP CODE COLUMNS


def json_to_df(json):
    """
    Convert a list of JSON objects containing location information into a Pandas DataFrame.

    :param json: A list of JSON objects containing location information.
    :return: A Pandas DataFrame containing the location information.
    """
    data = []

    for j in json:
        zipcode_ = j['census_zcta_geoid']
        for result in j['responce']['results']:
            try:
                # extracting the state from the compound code and check if it's 'GA'
                state_ = result['plus_code']['compound_code'].split(',')[-2].strip()
                poi_types_ = result['types']

                if state_ == 'GA':
                    # extracting relevant information from the JSON object
                    place_id_ = result['place_id']
                    latitude_ = result['geometry']['location']['lat']
                    longitude_ = result['geometry']['location']['lng']
                    rating_ = result.get('rating', None)
                    num_ratings_ = result.get('user_ratings_total', None)
                    name_ = result['name']
                    address_ = result['vicinity']

                    # appending the extracted information to the data list
                    data.append([latitude_, longitude_, state_, place_id_, name_, rating_, num_ratings_, poi_types_, zipcode_, address_])
            except:
                pass

    # creating df using the data list
    df = pd.DataFrame(data, columns=['latitude', 'longitude', 'state', 'place_id', 'name', 'rating', 'num_ratings', 'poi_types', 'zipcode', 'address'])
    return df


<IPython.core.display.Javascript object>

In [4]:
# # THIS CODE LOADS ONLY OBSERVATIONS WITH RATINGS

# def json_to_df(json):
#     """
#     Convert a list of JSON objects containing location information into a Pandas DataFrame.

#     :param json: A list of JSON objects containing location information.
#     :return: A Pandas DataFrame containing the location information.
#     """
#     data = []

#     for j in json:
#         zipcode_ = j['census_zcta_geoid']
#         for result in j['responce']['results']:
#             try:
#                 # extracting the state from the compound code and check if it's 'GA'
#                 state_ = result['plus_code']['compound_code'].split(',')[-2].strip()
#                 poi_types_ = result['types']

#                 if state_ == 'GA':
#                     # extracting relevant information from the JSON object
#                     place_id_ = result['place_id']
#                     latitude_ = result['geometry']['location']['lat']
#                     longitude_ = result['geometry']['location']['lng']
#                     rating_ = result.get('rating', None)
#                     num_ratings_ = result.get('user_ratings_total', None)
#                     name_ = result['name']
#                     address_ = result['vicinity']

#                     # check if the POI has user reviews
#                     if num_ratings_ is not None and num_ratings_ > 0:
#                         # appending the extracted information to the data list
#                         data.append([latitude_, longitude_, state_, place_id_, name_, rating_, num_ratings_, poi_types_, zipcode_, address_])
#             except:
#                 pass

#     # creating df using the data list
#     df = pd.DataFrame(data, columns=['latitude', 'longitude', 'state', 'place_id', 'name', 'rating', 'num_ratings', 'poi_types', 'zipcode', 'address'])
#     return df


<IPython.core.display.Javascript object>

In [5]:
grocery_df = json_to_df(grocery_json)

<IPython.core.display.Javascript object>

In [6]:
grocery_df.shape

(10174, 10)

<IPython.core.display.Javascript object>

In [7]:
grocery_df.isna().sum()

latitude          0
longitude         0
state             0
place_id          0
name              0
rating         6984
num_ratings    6984
poi_types         0
zipcode           0
address           0
dtype: int64

<IPython.core.display.Javascript object>

In [8]:
restaurant_df = json_to_df(restaurant_json)

<IPython.core.display.Javascript object>

In [9]:
restaurant_df.shape

(5432, 10)

<IPython.core.display.Javascript object>

In [10]:
poi_df = pd.concat([grocery_df, restaurant_df], axis=0)

<IPython.core.display.Javascript object>

In [11]:
# poi_df["primary_type"] = poi_df["poi_types"].apply(lambda x: x[0])

<IPython.core.display.Javascript object>

In [12]:
poi_df.isna().sum()

latitude          0
longitude         0
state             0
place_id          0
name              0
rating         7133
num_ratings    7133
poi_types         0
zipcode           0
address           0
dtype: int64

<IPython.core.display.Javascript object>

In [13]:
poi_df.head(3)

,latitude,longitude,state,place_id,name,rating,num_ratings,poi_types,zipcode,address
0,33.791987,-84.385677,GA,ChIJoz4--E8E9YgRxpSO8xC1jj0,Residence Inn Atlanta Midtown/Peachtree at 17th,4.1,723.0,"[lodging, point_of_interest, establishment]",30309,"1365 Peachtree Street Northeast, Atlanta"
1,33.790216,-84.388776,GA,ChIJh-lI8FoE9YgRyKGKYvLpvA8,Artmore Hotel,3.9,1106.0,"[lodging, point_of_interest, establishment]",30309,"1302 West Peachtree Street Northwest, Atlanta"
2,33.799427,-84.392589,GA,ChIJFybLXVQE9YgRTePNqN08VnU,Atlanta Peachtree Station,3.6,163.0,"[train_station, transit_station, point_of_inte...",30309,"1688 Peachtree Road Northwest, Atlanta"


<IPython.core.display.Javascript object>

In [14]:
# poi_df['primary_type'].value_counts()

<IPython.core.display.Javascript object>

In [15]:

# Create an S3 resource instead of using the S3 client that was used earlier
s3R = boto3.resource('s3')

# Convert the poi_df DataFrame to a csv buffer in memory
csv_buffer = io.StringIO()
poi_df.to_csv(csv_buffer, index=False)

# Naming the file for AWS storage
file_name = 'poi_combined_haystack_ALL.csv'

# Uploading the file to S3 and printing the response
response = s3R.Object(bucket_name, file_name).put(Body=csv_buffer.getvalue())
print(f"File uploaded to S3 with ETag: {response['ETag']}") # Checking the status to confirm it was uploaded


File uploaded to S3 with ETag: "89b136b22791ae4df5cd71e4b7efe810"


<IPython.core.display.Javascript object>